In [ ]:
!apt update -y
!apt install -y chromium-chromedriver
!pip install selenium webdriver-manager beautifulsoup4
#test


'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [13]:
!apt-get remove -y chromium-browser
!apt-get purge -y chromium-chromedriver
!apt-get autoremove -y
!rm -rf /usr/bin/chromedriver



'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt-get'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [14]:
!apt update
!apt install -y wget unzip
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt --fix-broken install -y


'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'dpkg'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'apt'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [15]:
!google-chrome --version


'google-chrome'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [16]:
CHROME_VERSION = "133"  # 위에서 확인한 Chrome 버전 앞 3자리 입력 (예: 120)

!wget https://chromedriver.storage.googleapis.com/{CHROME_VERSION}.0.6099.71/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!mv chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver


'wget'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'unzip'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'mv'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'chmod'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [17]:
# 최신 Chrome 버전 확인
!google-chrome --version

# Chrome 버전 자동 추출
import re
import subprocess

chrome_version = subprocess.run(["google-chrome", "--version"], capture_output=True, text=True).stdout
match = re.search(r"\d+\.\d+\.\d+\.\d+", chrome_version)
if match:
    CHROME_VERSION = match.group(0)
else:
    raise Exception("Chrome 버전을 찾을 수 없습니다.")

print(f"✅ Chrome Version Detected: {CHROME_VERSION}")

# Chrome 버전에 맞는 Chromedriver 다운로드
CHROMEDRIVER_URL = f"https://storage.googleapis.com/chrome-for-testing-public/{CHROME_VERSION}/linux64/chromedriver-linux64.zip"

!wget -O chromedriver.zip {CHROMEDRIVER_URL}
!unzip chromedriver.zip
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
!chmod +x /usr/bin/chromedriver


'google-chrome'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [21]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

# Chrome WebDriver 자동 설치
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

print("✅ Chrome WebDriver가 정상적으로 실행되었습니다.")
driver.quit()


✅ Chrome WebDriver가 정상적으로 실행되었습니다.


In [ ]:
!which chromedriver
!chromedriver --version


In [ ]:
#Jaderná elektrárna Dukovany(두코바니 원자력 발전소)

#https://search.seznam.cz/clanky/?q=Jadern%C3%A1%20elektr%C3%A1rna%20Dukovany

In [ ]:
import time
import json
import datetime
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

# 🔍 검색어 리스트
search_queries = ["두코바니 원전"]  # 원하는 검색어 입력

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2022.01.01&de=2024.12.31"

# ✅ 크롤링할 날짜 범위
START_DATE = datetime.datetime(2024, 9, 5)
END_DATE = datetime.datetime(2024, 12, 31)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정 (Chrome Headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())  # Chromedriver 자동 다운로드 및 경로 설정
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query):
    """Selenium을 사용하여 네이버 뉴스 검색 페이지 크롤링 (무한 스크롤)"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query)
    driver.get(search_url)
    time.sleep(2)  # 페이지 로딩 대기

    # ✅ 무한 스크롤을 통해 뉴스 목록 로드
    print(f"🔍 검색 중: {query} (스크롤 크롤링 시작)")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)  # 로딩 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 더 이상 로딩할 내용 없음
        last_height = new_height

    # ✅ BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            # ✅ 날짜 가져오기 (목록에서 직접 크롤링)
            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 링크 및 날짜를 수집 완료!")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        try:
            response = requests.get(news["link"], headers=HEADERS)
            soup = BeautifulSoup(response.text, "html.parser")

            # ✅ 본문 크롤링 (다양한 선택자 추가)
            content_selectors = [
                "div#newsct_article",  # 최신 네이버 뉴스 본문
                "div#articleBodyContents",  # 과거 네이버 뉴스 본문
                "div.article_view",
                "div.news_end",
                "div#articeBody",
                "div.content_area",
                "div#newsEndContents",
                "article"
            ]
            content = ""
            for selector in content_selectors:
                content_element = soup.select(selector)
                if content_element:
                    content = " ".join([p.text.strip() for p in content_element])
                    break

            # ✅ Fallback: Selenium으로 직접 본문 가져오기
            if not content.strip():
                print(f"⚠ 본문을 가져오지 못해 Selenium으로 재시도: {news['link']}")
                driver.get(news["link"])
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, "html.parser")
                content_element = soup.select_one("div#newsct_article") or soup.select_one("article")
                if content_element:
                    content = " ".join([p.text.strip() for p in content_element])

            # ✅ 기사 본문에서 날짜 확인 (목록과 비교)
            meta_date_element = soup.select_one("meta[property='article:published_time']")
            if meta_date_element:
                article_date = meta_date_element["content"][:10].replace("-", ".")  # 'YYYY-MM-DD' -> 'YYYY.MM.DD'
                news["date"] = article_date  # 목록에서 가져온 날짜를 덮어씀

            # ✅ 데이터 저장
            news["content"] = content.strip() if content else "N/A"
            news_data.append(news)

            print(f"✅ 크롤링 완료: {news['title'][:50]}...")

            time.sleep(1)  # 서버 부하 방지

        except Exception as e:
            print(f"⚠ 오류 발생 (뉴스 크롤링 실패): {news['link']}, 오류: {e}")

    return news_data

# ✅ 전체 검색어에 대해 뉴스 크롤링 실행
all_news = []
for query in search_queries:
    news_list = fetch_naver_news(query)
    news_data = fetch_news_content(news_list)
    all_news.extend(news_data)

# ✅ JSON 파일로 저장
output_file = "naver_news.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_news, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(all_news)}개의 뉴스 기사를 저장했습니다. (파일명: {output_file})")

# ✅ Selenium 종료
driver.quit()


In [ ]:
import time
import json
import datetime
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

# 🔍 검색어 리스트
search_queries = ["두코바니 원전"]  # 원하는 검색어 입력

# ✅ 네이버 뉴스 검색 URL 템플릿
NAVER_NEWS_URL_TEMPLATE = "https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2022.01.01&de=2024.12.31"

# ✅ 크롤링할 날짜 범위
START_DATE = datetime.datetime(2022, 1, 1)
END_DATE = datetime.datetime(2024, 12, 31)

# ✅ User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
}

# ✅ Selenium 옵션 설정 (Chrome Headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUI 없이 실행
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

# ✅ ChromeDriver 자동 설치
service = Service(ChromeDriverManager().install())  # Chromedriver 자동 다운로드 및 경로 설정
driver = webdriver.Chrome(service=service, options=chrome_options)

def fetch_naver_news(query):
    """Selenium을 사용하여 네이버 뉴스 검색 페이지 크롤링 (무한 스크롤)"""
    search_url = NAVER_NEWS_URL_TEMPLATE.format(query=query)
    driver.get(search_url)
    time.sleep(2)  # 페이지 로딩 대기

    # ✅ 무한 스크롤을 통해 뉴스 목록 로드
    print(f"🔍 검색 중: {query} (스크롤 크롤링 시작)")
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        time.sleep(2)  # 로딩 대기
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 더 이상 로딩할 내용 없음
        last_height = new_height

    # ✅ BeautifulSoup으로 페이지 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("div.group_news > ul.list_news > li")

    news_list = []
    for article in articles:
        try:
            title_element = article.select_one("a.news_tit")
            title = title_element.text.strip() if title_element else "N/A"
            link = title_element["href"] if title_element else "N/A"

            # ✅ 날짜 가져오기 (목록에서 직접 크롤링)
            date_element = article.select_one("span.info")
            date_text = date_element.text.strip() if date_element else "N/A"

            news_list.append({"title": title, "link": link, "date": date_text})

        except Exception as e:
            print(f"⚠ 오류 발생 (목록 크롤링 실패): {e}")

    print(f"✅ {len(news_list)}개의 뉴스 기사 링크 및 날짜를 수집 완료!")
    return news_list

def fetch_news_content(news_list):
    """뉴스 기사 본문 크롤링"""
    news_data = []

    for news in tqdm(news_list, desc="📰 뉴스 본문 크롤링 진행"):
        try:
            response = requests.get(news["link"], headers=HEADERS)
            soup = BeautifulSoup(response.text, "html.parser")

            # ✅ 본문 크롤링 (다양한 선택자 추가)
            content_selectors = [
                "div#newsct_article",  # 최신 네이버 뉴스 본문
                "div#articleBodyContents",  # 과거 네이버 뉴스 본문
                "div.article_view",
                "div.news_end",
                "div#articeBody",
                "div.content_area",
                "div#newsEndContents",
                "article"
            ]
            content = ""
            for selector in content_selectors:
                content_element = soup.select(selector)
                if content_element:
                    content = " ".join([p.text.strip() for p in content_element])
                    break

            # ✅ Fallback: Selenium으로 직접 본문 가져오기
            if not content.strip():
                print(f"⚠ 본문을 가져오지 못해 Selenium으로 재시도: {news['link']}")
                driver.get(news["link"])
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, "html.parser")
                content_element = soup.select_one("div#newsct_article") or soup.select_one("article")
                if content_element:
                    content = " ".join([p.text.strip() for p in content_element])

            # ✅ 기사 본문에서 날짜 확인 (목록과 비교)
            meta_date_element = soup.select_one("meta[property='article:published_time']")
            if meta_date_element:
                article_date = meta_date_element["content"][:10].replace("-", ".")  # 'YYYY-MM-DD' -> 'YYYY.MM.DD'
                news["date"] = article_date  # 목록에서 가져온 날짜를 덮어씀

            # ✅ 데이터 저장
            news["content"] = content.strip() if content else "N/A"
            news_data.append(news)

            print(f"✅ 크롤링 완료: {news['title'][:50]}...")

            time.sleep(1)  # 서버 부하 방지

        except Exception as e:
            print(f"⚠ 오류 발생 (뉴스 크롤링 실패): {news['link']}, 오류: {e}")

    return news_data

# ✅ 전체 검색어에 대해 뉴스 크롤링 실행
all_news = []
for query in search_queries:
    news_list = fetch_naver_news(query)
    news_data = fetch_news_content(news_list)
    all_news.extend(news_data)

# ✅ JSON 파일로 저장
output_file = "naver_news.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_news, f, ensure_ascii=False, indent=4)

print(f"✅ 총 {len(all_news)}개의 뉴스 기사를 저장했습니다. (파일명: {output_file})")

# ✅ Selenium 종료
driver.quit()


🔍 검색 중: 두코바니 원전 (스크롤 크롤링 시작)
